In [2]:
!pip install geopy
!pip install pandas
!pip install pathos
!pip install xgboost
!pip install azure-storage
!pip install reverse_geocoder

    100% |################################| 71kB 2.4MB/s ta 0:00:01
    100% |################################| 92kB 2.8MB/s ta 0:00:011
    100% |################################| 81kB 3.8MB/s ta 0:00:011
    100% |################################| 71kB 4.7MB/s ta 0:00:011
    100% |################################| 51kB 6.0MB/s ta 0:00:011
    100% |################################| 1.5MB 806kB/s eta 0:00:01
  Running setup.py bdist_wheel for pathos ... - done
  Stored in directory: /home/nbuser/.cache/pip/wheels/6b/b1/41/c44a53e7e20d98578cf5c14b7aac72822c6416f6439a45d349
  Running setup.py bdist_wheel for ppft ... - done
  Stored in directory: /home/nbuser/.cache/pip/wheels/4d/28/cc/c5bca1e1d3923a5f4dd50219a9e41391bbeb832835e7905344
  Running setup.py bdist_wheel for dill ... - done
  Stored in directory: /home/nbuser/.cache/pip/wheels/e5/88/fe/7e290ce5bb39d531eb9bee5cf254ba1c3e3c7ba3339ce67bee
  Running setup.py bdist_wheel for pox ... - done
  Stored in directory: /home/nb

In [3]:
import pandas as pd
from azure.storage.file import FileService
import os

account_name = 'kagglenyctaxi'
account_key = '9dX6B7LpElaR7rBBicZUmdUM5f5LkoT6TrWZd4rMRno9IP5BwVN0HojpSt90kHGTsMFMUaSEIxjs8LBbd7zw6g=='
file_service = FileService(account_name=account_name, account_key=account_key)
def get_train_data():
    file_service.get_file_to_path('data', None, 'train.csv', 'train.csv')
def get_test_data():
    file_service.get_file_to_path('data', None, 'test.csv', 'test.csv')

if not os.path.exists('train.csv'):
    get_train_data()
if not os.path.exists('test.csv'):
    get_test_data()
dataframe = pd.read_csv('train.csv', index_col=0)
test_dataframe = pd.read_csv('test.csv', index_col=0)
print dataframe.head()

           vendor_id      pickup_datetime     dropoff_datetime  \
id                                                               
id2875421          2  2016-03-14 17:24:55  2016-03-14 17:32:30   
id2377394          1  2016-06-12 00:43:35  2016-06-12 00:54:38   
id3858529          2  2016-01-19 11:35:24  2016-01-19 12:10:48   
id3504673          2  2016-04-06 19:32:31  2016-04-06 19:39:40   
id2181028          2  2016-03-26 13:30:55  2016-03-26 13:38:10   

           passenger_count  pickup_longitude  pickup_latitude  \
id                                                              
id2875421                1        -73.982155        40.767937   
id2377394                1        -73.980415        40.738564   
id3858529                1        -73.979027        40.763939   
id3504673                1        -74.010040        40.719971   
id2181028                1        -73.973053        40.793209   

           dropoff_longitude  dropoff_latitude store_and_fwd_flag  \
id          

In [4]:
from geopy.distance import great_circle
from dateutil.parser import parse
import time
from pathos.multiprocessing import ProcessingPool as Pool
from pathos.multiprocessing import cpu_count
import numpy as np

# def get_borough(lat, lng, retries=3):
#     if retries <= -1: return None
#     try:
#         geolocator = Nominatim()
#         location = geolocator.reverse('{}, {}'.format(lat, lng))
#         return location.address.split(', ')[2]
#     except Exception as e:
#         print 'Too many requests. Waiting for 2 mins for {}, {}'.format(lat, lng)
#         time.sleep(30)
#         return get_borough(lat, lng, retries-1)
        
def transform_df(df, great_circle=great_circle, parse=parse, np=np):
    yes_func = lambda x: 1 if x == 'Y' else 0
    df['distance'] = df.apply(lambda row : great_circle((row['pickup_latitude'], row['pickup_longitude']), 
                                                        (row['dropoff_latitude'], row['dropoff_longitude'])).miles, axis=1)
                              
    df['month'] = df.apply(lambda row: parse(row['pickup_datetime']).month, axis=1)
    df['day'] = df.apply(lambda row: parse(row['pickup_datetime']).weekday(), axis=1)
    df['pickup_hour'] = df.apply(lambda row: parse(row['pickup_datetime']).hour, axis=1)
    df['store_and_fwd_flag'] = df.apply(lambda row: yes_func(row['store_and_fwd_flag']), axis=1)
    return df

def parallelize_dataframe(df, func, num_cores, num_partitions):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.clear()
    return df

new_df = parallelize_dataframe(dataframe, transform_df, cpu_count(), cpu_count())
new_df['trip_duration'] = new_df.apply(lambda row: np.log1p(row['trip_duration']), axis=1)
print new_df.head()

           vendor_id      pickup_datetime     dropoff_datetime  \
id                                                               
id2875421          2  2016-03-14 17:24:55  2016-03-14 17:32:30   
id2377394          1  2016-06-12 00:43:35  2016-06-12 00:54:38   
id3858529          2  2016-01-19 11:35:24  2016-01-19 12:10:48   
id3504673          2  2016-04-06 19:32:31  2016-04-06 19:39:40   
id2181028          2  2016-03-26 13:30:55  2016-03-26 13:38:10   

           passenger_count  pickup_longitude  pickup_latitude  \
id                                                              
id2875421                1        -73.982155        40.767937   
id2377394                1        -73.980415        40.738564   
id3858529                1        -73.979027        40.763939   
id3504673                1        -74.010040        40.719971   
id2181028                1        -73.973053        40.793209   

           dropoff_longitude  dropoff_latitude  store_and_fwd_flag  \
id         

In [11]:
import reverse_geocoder as rg
from sklearn import preprocessing

# def get_borough_rev_geocoder(lat, lng):
#     return rg.search((lat, lng))[0]['name']
# dataframe['pickup_borough'] = dataframe.apply(lambda row: 
#                                               get_borough_rev_geocoder(row['pickup_latitude'], row['pickup_longitude']), axis=1)
def get_coords(df, lat_key, long_key):
    lats = df[lat_key].values.tolist()
    longs = df[long_key].values.tolist()
    coords = zip(lats, longs)
    return coords
def encode_labels(labels):
    le = preprocessing.LabelEncoder()
    le.fit(labels)
    return le
def add_borough(df):
    pickup_coords = get_coords(df, 'pickup_latitude', 'pickup_longitude')
    dropoff_coords = get_coords(df, 'dropoff_latitude', 'dropoff_longitude')
    pickup_boroughs = np.array([d['admin2'] for d in rg.search(pickup_coords)])
    dropoff_boroughs = np.array([d['admin2'] for d in rg.search(dropoff_coords)])
    df['pickup_borough'] = encode_labels(pickup_boroughs).transform(pickup_boroughs)
    df['dropoff_borough'] = encode_labels(dropoff_boroughs).transform(dropoff_boroughs)
    return df
new_df = add_borough(new_df)
print new_df.head()

           vendor_id      pickup_datetime     dropoff_datetime  \
id                                                               
id2875421          2  2016-03-14 17:24:55  2016-03-14 17:32:30   
id2377394          1  2016-06-12 00:43:35  2016-06-12 00:54:38   
id3858529          2  2016-01-19 11:35:24  2016-01-19 12:10:48   
id3504673          2  2016-04-06 19:32:31  2016-04-06 19:39:40   
id2181028          2  2016-03-26 13:30:55  2016-03-26 13:38:10   

           passenger_count  pickup_longitude  pickup_latitude  \
id                                                              
id2875421                1        -73.982155        40.767937   
id2377394                1        -73.980415        40.738564   
id3858529                1        -73.979027        40.763939   
id3504673                1        -74.010040        40.719971   
id2181028                1        -73.973053        40.793209   

           dropoff_longitude  dropoff_latitude  store_and_fwd_flag  \
id         

In [12]:
features = ['vendor_id', 'month', 'day', 'pickup_hour', 'store_and_fwd_flag', 'distance', 'pickup_borough', 'dropoff_borough']
def scale_df(df, test=False):
    if test:
        customized_df = df[features]
        x = customized_df.values #returns a numpy array
    else:
        customized_df = df[features + ['trip_duration']]
        x = customized_df.values[:,:-1] #returns a numpy array
        

    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    if not test:
        trip_duration_col = np.array([customized_df['trip_duration'].values])
        x_scaled = np.concatenate((x_scaled, trip_duration_col.T), 1)
    scaled_df = pd.DataFrame(x_scaled, columns=customized_df.columns)
    return scaled_df
scaled_df = scale_df(new_df)
print scaled_df.head()

   vendor_id  month       day  pickup_hour  store_and_fwd_flag  distance  \
0        1.0    0.4  0.000000     0.739130                 0.0  0.001208   
1        0.0    1.0  1.000000     0.000000                 0.0  0.001455   
2        1.0    0.0  0.166667     0.478261                 0.0  0.005146   
3        1.0    0.6  0.333333     0.826087                 0.0  0.001197   
4        1.0    0.4  0.833333     0.565217                 0.0  0.000958   

   pickup_borough  dropoff_borough  trip_duration  
0        0.604651         0.652174       6.122493  
1        0.720930         0.000000       6.498282  
2        0.604651         0.000000       7.661527  
3        0.000000         0.000000       6.063785  
4        0.604651         0.652174       6.077642  


In [13]:
from sklearn.ensemble import GradientBoostingRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

def rlmse_func(predicted, actual):
    return np.sqrt(np.mean(np.square(np.log(predicted+1.0) - np.log(actual+1.0))))

rlmse = make_scorer(rlmse_func, greater_is_better=False)

# param_grid = {'learning_rate': [0.1, 0.05, 0.02, 0.01],
#               'max_depth': [4, 6],
#               'min_samples_leaf': [3, 5, 9, 17],
#               }

X_train, y_train = scaled_df[features].values, scaled_df[['trip_duration']].values
xgb_model = XGBRegressor(objective='reg:linear', max_depth=5, learning_rate=0.3, n_estimators=2000, nthread=-1)
print -1.0*cross_val_score(xgb_model, X_train, y_train.ravel(), scoring=rlmse, cv=10, verbose=8).mean()

# gs_cv = GridSearchCV(model, param_grid, n_jobs=4, verbose=4, scoring=rmse).fit(X_train, y_train)
# print gs_cv.best_params_
# print gs_cv.best_score_

/home/nbuser/anaconda2_410/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/nbuser/anaconda2_410/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


[CV]  ................................................................
[CV] ................................ , score=-0.065392, total= 7.9min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.9min remaining:    0.0s


[CV] ................................ , score=-0.066165, total= 6.9min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 14.7min remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 43.0min remaining:    0.0s


[CV] ................................ , score=-0.065288, total= 7.0min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed: 50.0min remaining:    0.0s


[CV] ................................ , score=-0.065818, total= 7.9min
[CV]  ................................................................
[CV] ................................ , score=-0.065837, total= 7.0min
[CV]  ................................................................
[CV] ................................ , score=-0.065744, total= 6.9min
0.0657493933673


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 71.9min finished


In [14]:
test_df = parallelize_dataframe(test_dataframe, transform_df, cpu_count(), cpu_count())
new_test_df = scale_df(add_borough(test_df), test=True)
print new_test_df.head()

   vendor_id  month  day  pickup_hour  store_and_fwd_flag  distance  \
0        0.0    1.0  0.5          1.0                 0.0  0.002651   
1        0.0    1.0  0.5          1.0                 0.0  0.002663   
2        0.0    1.0  0.5          1.0                 0.0  0.001261   
3        1.0    1.0  0.5          1.0                 0.0  0.005086   
4        0.0    1.0  0.5          1.0                 0.0  0.000927   

   pickup_borough  dropoff_borough  
0        0.000000         0.333333  
1        0.310345         0.366667  
2        0.000000         0.000000  
3        0.586207         0.000000  
4        0.586207         0.700000  


In [16]:
xgb_model.fit(X_train, y_train)
predictions = xgb_model.predict(new_test_df.values)
print predictions

[ 8.15130615  7.105124    6.1855197  ...,  6.50277948  9.21678448
  7.09972954]


In [17]:
test_trip_duration = np.expm1(predictions)
result_df = pd.DataFrame({'id': test_dataframe.index.values, 'trip_duration': test_trip_duration})
result_df.to_csv('answer.csv', index=False)